In [ ]:
from datasets import load_dataset
import json

# Load a dataset (replace with your custom dataset)
dataset = load_dataset("openai/math")

def format_data(example):
    """
    Formats dataset into (problem, CoT solution) pairs.
    """
    return {
        "problem": example["question"],
        "solution": example["solution"]
    }

# Process dataset
formatted_dataset = dataset.map(format_data)
formatted_dataset.save_to_disk("formatted_math_dataset")


In [9]:
import openai
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()
client = openai.OpenAI(api_key=api_key)

# Access the API key
api_key = os.getenv("OPENAI_API_KEY")
# List all available models
models = client.models.list()

# Print model names
# for model in models:
#     print(model.id)

def generate_tir_solution(problem):
    """
    Uses GPT-4 to generate a Tool-Integrated Reasoning (TIR) solution.
    """
    client = openai.OpenAI(api_key=api_key)

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": "Generate a step-by-step solution for the math problem, with Python code for execution."},
            {"role": "user", "content": problem}
        ],
        temperature=0.7
    )
    return response["choices"][0]["message"]["content"]

# Example usage
problem = "Find the determinant of the matrix [[2,3],[4,5]]."
solution = generate_tir_solution(problem)
print(solution)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
def execute_code(code):
    """
    Executes the generated Python code safely.
    """
    try:
        exec_globals = {}
        exec(code, {}, exec_globals)
        return exec_globals.get("result", None)
    except Exception as e:
        return f"Error: {str(e)}"

# Example usage
code_snippet = "result = 2*5 - 3*4"
print(execute_code(code_snippet))


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer

# Load model and tokenizer
model_name = "deepseek-ai/deepseek-math-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Tokenize dataset
def tokenize_function(example):
    return tokenizer(example["problem"] + "\n" + example["solution"], truncation=True, padding="max_length", max_length=2048)

train_dataset = formatted_dataset["train"].map(tokenize_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="math_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    num_train_epochs=3,
    fp16=True,
    logging_steps=50,
    save_total_limit=2
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

# Train the model
trainer.train()


In [ ]:
import torch
from transformers import pipeline

# Load fine-tuned model
model = AutoModelForCausalLM.from_pretrained("math_model")
tokenizer = AutoTokenizer.from_pretrained("math_model")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

def generate_solutions(problem, num_samples=48):
    """
    Generate multiple solutions using self-consistency.
    """
    outputs = pipe(problem, max_length=2048, num_return_sequences=num_samples, do_sample=True, temperature=0.7)
    return [output["generated_text"] for output in outputs]

# Example
problem = "Solve for x: 3x + 5 = 20."
solutions = generate_solutions(problem)
print(solutions[:3])  # Show first 3 solutions


In [ ]:
from collections import Counter

def majority_voting(solutions):
    """
    Select the most common valid answer from generated solutions.
    """
    filtered_solutions = [execute_code(solution) for solution in solutions if execute_code(solution) is not None]
    return Counter(filtered_solutions).most_common(1)[0][0] if filtered_solutions else "No valid solution"

# Get the final answer
final_answer = majority_voting(solutions)
print("Final Answer:", final_answer)
